# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they are the best equipped to handle statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
"context": "The task involves tracking and analyzing market data for the selected stock, AAPL. This includes identifying trends and predicting market movements using statistical modeling and machine learning techniques."
}


> Entering new CrewAgentExecutor chain...
I need to start by gathering the latest market data for AAPL to anal

Final Answer: 
After analyzing the market data for AAPL from Yahoo Finance, the stock closed at $222.77, with a decrease of 2.72%. The market cap stands at 3.387 trillion. The company, Apple Inc., specializes in consumer electronics and technology, offering a range of products and services. The stock has shown various returns over different periods, with a YTD return of 16.15% and a 1-year return of 18.18%. Additionally, key financial metrics such as profit margin, return on assets, and revenue are provided. Overall, the analysis indicates a comprehensive overview of AAPL's performance in the market.

> Finished chain.
 

After analyzing the market data for AAPL from Yahoo Finance, the stock closed at $222.77, with a decrease of 2.72%. The market cap stands at 3.387 trillion. The company, Apple Inc., specializes in consumer electronics and technology, offering a range of products and services. The stock has shown various returns over different periods, with a YTD return of 16.15% and a

Final Answer: Based on the insights gathered from analyzing the market data for AAPL and the information obtained from Yahoo Finance, a potential trading strategy for AAPL could involve considering the company's strong financial performance, market position, and product offerings. With a market cap of 3.387 trillion and a history of consistent returns over different periods, AAPL presents itself as a stable and lucrative investment opportunity. Considering the profit margin of 26.44%, return on assets of 22.61%, and revenue of $385.6 billion, it is evident that AAPL is a financially robust company.

To develop a trading strategy for AAPL, it is important to consider the company's upcoming earnings dates, forward dividend yield, and historical stock performance. Additionally, analyzing the stock's price movements, trading volume, and technical indicators can provide further insights into potential entry and exit points for trading AAPL.

Furthermore, keeping an eye on key events, corpor

I need to gather more information on approved trading strategies for AAPL to provide the best answer to my coworker.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/articles/active-trading/080715/4-best-strategies-day-trading.asp"} 

Page Not Found
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Derivatives
 Commodities
 Trading
 Automated Investing
 Brokers
 Fundamental Analysis
 Markets
 View All
 Simulator
 Login / Portfolio
 Trade
 Research
 My Games
 Leaderboard
 Banking
 Savings Accounts
 Certificates of Deposit (CDs)
 Money Market Accounts
 Checking Accounts
 View All
 Personal Finance
 Budgeting and Saving
 Personal Loans
 Insurance
 Mortgages
 Credit and Debt
 Student Loans
 Taxes
 Credit Cards
 Financial Literacy
 Retirement
 View All
 Economy
 Government and Policy
 Monetary Policy
 Fiscal Policy
 Economics
 View All
 News
 Markets
 Companies
 Earnings
 CD Rates
 Mortgage Rates
 Economy
 Government
 Crypto
 ETFs
 Persona

Final Answer: 
Based on the information gathered from Yahoo Finance, the current market conditions for AAPL show a closing price of $222.77 with a decrease of 2.72%. The forward dividend yield is 0.44% with an ex-dividend date of August 12, 2024. The earnings date is scheduled between October 31, 2024, and November 4, 2024. Additionally, the historical stock performance indicates a YTD return of 15.71% and a 5-year return of 326.89%. Other key metrics such as market cap, beta, PE ratio, EPS, and revenue are also provided for further analysis.

> Finished chain.
 

Based on the information gathered from Yahoo Finance, the current market conditions for AAPL show a closing price of $222.77 with a decrease of 2.72%. The forward dividend yield is 0.44% with an ex-dividend date of August 12, 2024. The earnings date is scheduled between October 31, 2024, and November 4, 2024. Additionally, the historical stock performance indicates a YTD return of 15.71% and a 5-year return of 326.89%. Other 

Final Answer: 
After analyzing Apple Inc.'s financial performance and market dynamics, I have identified several key risk factors associated with the proposed trading strategies for AAPL:

1. Market Volatility: The stock price movements of AAPL can be influenced by market volatility, which may lead to unexpected fluctuations in the stock price.

2. Earnings Date: The upcoming earnings dates for AAPL, scheduled between October 31, 2024, and November 4, 2024, could impact the stock price based on the company's performance.

3. Forward Dividend Yield: The forward dividend yield of 0.44% may not provide significant returns for investors seeking income from dividends.

4. Historical Stock Performance: While AAPL has shown strong historical stock performance with a 5-year return of 326.89%, past performance may not guarantee future results.

5. External Factors: External factors such as market trends, corporate changes, and key events can also affect AAPL's stock performance.

To mitigate th

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for AAPL:

After evaluating the risks associated with the proposed trading strategies and execution plans for AAPL, several key risk factors were identified:

1. Market Volatility: The stock price movements of AAPL can be influenced by market volatility, leading to unexpected fluctuations in the stock price.

2. Earnings Date Impact: The upcoming earnings dates for AAPL, scheduled between October 31, 2024, and November 4, 2024, could impact the stock price based on the company's performance during that period.

3. Forward Dividend Yield: The forward dividend yield of 0.44% may not provide significant returns for investors seeking income from dividends.

4. Historical Stock Performance: While AAPL has shown strong historical stock performance with a 5-year return of 326.89%, past performance may not guarantee future results.

5. External Factors: Market trends, corporate changes, and key events can also affect the stock performance of AAPL.

To mitigate these risks and ensure a well-rounded trading strategy, the following mitigation strategies are recommended:

1. Diversification: Diversify the investment portfolio to reduce exposure to individual stock risk and enhance overall risk-adjusted returns.

2. Risk Management: Implement a disciplined risk management approach to monitor and control potential losses, including setting stop-loss orders and implementing hedging strategies.

3. Stay Informed: Stay informed about the latest news, press releases, SEC filings, and market trends to make informed investment decisions and adapt trading strategies accordingly.

By incorporating these risk factors and mitigation strategies into the trading approach for AAPL, investors can optimize their trading strategies, minimize potential risks, and maximize their chances of success in the market.